In [1]:
import torch
from torch.nn import functional as F 
from torch import nn 
from torch import optim
from torchvision import datasets,transforms


# training data

In [2]:
batch_size = 200 #每次的樣本數量
learning_rate=0.01 
epochs = 10 
# 讀取MNIST手寫數字數據集,初次運行下載到../data/目錄下

# 要注意所有樣本進行標準化的參數要保持一致(這裡是樣本和方差)
# 這個標準化的參數是數據提供方計算好的
# 所以就不用自己計算了,在網上查好然後標準化時候寫進去就可以了
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',train=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(), #轉成tensor的類型
                       transforms.Normalize((0.1307,),(0.3081,)) 
                   ])),
    batch_size=batch_size,shuffle=True)

C:\Users\sam\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# testing data

In [3]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(), #轉成tensor的類型
                       transforms.Normalize((0.1307,),(0.3081,)) 
                   ])),
    batch_size=batch_size,shuffle=True)
test_loader

# linear layer

In [19]:
#input為784 out為200
w1 = torch.randn(200,784,requires_grad=True)
b1 = torch.randn(200,requires_grad=True)
w2 = torch.randn(200,200,requires_grad=True)
b2 = torch.randn(200,requires_grad=True)
w3 = torch.randn(10,200,requires_grad=True)
b3 = torch.randn(10,requires_grad=True)
#下面這三行很重要 若沒初始化可能會梯度消失
torch.nn.init.kaiming_normal_(w1)
torch.nn.init.kaiming_normal_(w2)
torch.nn.init.kaiming_normal_(w3)

tensor([[-0.1362, -0.1365, -0.0645,  ..., -0.1168,  0.0714,  0.0454],
        [ 0.0589,  0.0033, -0.1465,  ..., -0.0485,  0.0916, -0.1087],
        [ 0.0681,  0.1995, -0.0518,  ..., -0.1379,  0.0740,  0.0400],
        ...,
        [ 0.1376,  0.0130,  0.0884,  ...,  0.0568,  0.2364, -0.2266],
        [ 0.0762,  0.1269, -0.0058,  ...,  0.0180,  0.0333, -0.0910],
        [-0.0132,  0.0129,  0.1586,  ...,  0.0264,  0.1072,  0.1041]],
       requires_grad=True)

# linear function

In [20]:
def forward(x): #x 的matrix為[樣本數,784]
    x=x@w1.t()+b1
    x=F.relu(x)
    x=x@w2.t()+b2 
    x=F.relu(x)
    x=x@w3.t()+b3 
    x=F.relu(x)
    return x

#  選擇LOSS 跟GD

In [21]:
optimizer=optim.SGD([w1,b1,w2,b2,w3,b3],lr=1e-3)
CEL = nn.CrossEntropyLoss()


# training model

只有在資料很龐大的時候（在機器學習中，資料一般情況下都會很大），我們才需要使用epochs，batch size，iteration這些術語，在這種情況下，一次性將資料輸入計算機是不可能的。因此，為了解決這個問題，我們需要把資料分成小塊，一塊一塊的傳遞給計算機，在每一步的末端更新神經網路的權重，擬合給定的資料。


# epoch

當一個完整的資料集通過了神經網路一次並且返回了一次，這個過程稱為一次epoch。然而，當一個epoch對於計算機而言太龐大的時候，就需要把它分成多個小塊。
在神經網路中傳遞完整的資料集一次是不夠的，而且我們需要將完整的資料集在同樣的神經網路中傳遞多次。但請記住，我們使用的是有限的資料集，並且我們使用一個迭代過程即梯度下降來優化學習過程。

### 隨著epoch數量增加，神經網路中的權重的更新次數也在增加，曲線從欠擬合變得過擬合。對於不同的資料集，epoch的數量答案是不一樣的。 

In [26]:
for epoch in range(epochs):
#training
    for batch_index,(data,target) in enumerate(train_loader):
        #將data攤平成[樣本數,784]
        data=data.reshape(-1,28*28)
        logits=forward(data) #計算logits
        loss=CEL(logits,target) #計算Loss，且不需要softmax因為pytorch的cel已經有了
        optimizer.zero_grad() #清空梯度
        loss.backward()
        optimizer.step()
        if batch_index%100 ==0:
            print('Train Epoch: {}[{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(
            epoch,batch_index*len(data),len(train_loader.dataset),
            100.*batch_index/len(train_loader),loss.item()))
#   testing  
    test_loss=0
    correct=0
    for data , target in test_loader:
        data=data.reshape(-1,28*28)
        logits=forward(data)
        test_loss+=CEL(logits,target).item()
        #得到的預測值0~9 10個vector的機率，在第二個為杜取max
        #logits.data是一個shape=[batch_size,10]的Tensor
        #第一個tensor存的是最大值的值，第二個tensor是對應的索引
        pred=logits.data.max(dim=1)[1]
        correct+=pred.eq(target.data).sum()
    test_loss/=len(test_loader.dataset)
    print('\nTset set:Average loss:{:.4f},Accuracy:{}/{}({:.0f}%)\n'.format(
        test_loss,correct,len(test_loader.dataset),
        100.*correct/len(test_loader.dataset)))

Train Epoch: 0[0/60000 (0%)]	Loss:0.649469
Train Epoch: 0[20000/60000 (33%)]	Loss:0.639419
Train Epoch: 0[40000/60000 (67%)]	Loss:0.569756

Tset set:Average loss:0.0029,Accuracy:8177/10000(82%)

Train Epoch: 1[0/60000 (0%)]	Loss:0.564435
Train Epoch: 1[20000/60000 (33%)]	Loss:0.583145
Train Epoch: 1[40000/60000 (67%)]	Loss:0.392156

Tset set:Average loss:0.0028,Accuracy:8232/10000(82%)

Train Epoch: 2[0/60000 (0%)]	Loss:0.485450
Train Epoch: 2[20000/60000 (33%)]	Loss:0.540783
Train Epoch: 2[40000/60000 (67%)]	Loss:0.457254

Tset set:Average loss:0.0027,Accuracy:8257/10000(83%)

Train Epoch: 3[0/60000 (0%)]	Loss:0.695377
Train Epoch: 3[20000/60000 (33%)]	Loss:0.497880
Train Epoch: 3[40000/60000 (67%)]	Loss:0.575176

Tset set:Average loss:0.0027,Accuracy:8271/10000(83%)

Train Epoch: 4[0/60000 (0%)]	Loss:0.454336
Train Epoch: 4[20000/60000 (33%)]	Loss:0.545366
Train Epoch: 4[40000/60000 (67%)]	Loss:0.489399

Tset set:Average loss:0.0026,Accuracy:8285/10000(83%)

Train Epoch: 5[0/60000 (0

# testing


Tset set:Average loss:0.0043,Accuracy:7217/10000(72%)

